<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# SQL Server 2019 big data cluster Tutorial
## 02 - Data Virtualization

In this tutorial you will learn how to create and query Virtualized Data in a SQL Server big data cluster. 

**TODO:** Complete Tutorial 


In [31]:
/* Create External File Format */

USE WideWorldImporters;
GO

IF NOT EXISTS(SELECT * FROM sys.external_file_formats WHERE name = 'csv_file')
BEGIN
    CREATE EXTERNAL FILE FORMAT csv_file
    WITH (
        FORMAT_TYPE = DELIMITEDTEXT,
        FORMAT_OPTIONS(
            FIELD_TERMINATOR = ',',
            STRING_DELIMITER = '0x22',
            FIRST_ROW = 2,
            USE_TYPE_DEFAULT = TRUE)
    );
END

In [6]:
/* Create External Data Source to the Storage Pool */
IF NOT EXISTS(SELECT * FROM sys.external_data_sources WHERE name = 'SqlStoragePool')
BEGIN
  IF (SELECT @@VERSION) LIKE '%CTP2.3%'
    CREATE EXTERNAL DATA SOURCE SqlStoragePool
    WITH (LOCATION = 'sqlhdfs://service-mssql-controller:8080');
  ELSE IF (SELECT @@VERSION) LIKE '%CTP2.4%'
    CREATE EXTERNAL DATA SOURCE SqlStoragePool
    WITH (LOCATION = 'sqlhdfs://service-master-pool:50070');
END

In [40]:
/* Create an External Table that can read from the Storage Pool File Location */
IF NOT EXISTS(SELECT * FROM sys.external_tables WHERE name = 'web_customers_hdfs')
BEGIN
    CREATE EXTERNAL TABLE [web_customers_hdfs]
    ("CustomerCategoryName" VARCHAR(250) 
    , "PrimaryContact" VARCHAR(250) 
    , "WebsiteURL" VARCHAR(250))
    WITH
    (
        DATA_SOURCE = SqlStoragePool,
        LOCATION = '/adventureworks_export',
        FILE_FORMAT = csv_file
    );
END

In [2]:
/* Read Data from HDFS using only T-SQL */

SELECT hdfs.PrimaryContact
, hdfs.CustomerCategoryName
    FROM [web_customers_hdfs] hdfs
WHERE WebsiteURL LIKE '%wingtip%';
GO


(201 rows affected)

Total execution time: 00:00:01.8202476

PrimaryContact,CustomerCategoryName
Mario Jogi,Novelty Shop
Kaya ozbek,Novelty Shop
Seo-yeon Rim,Novelty Shop
Sitara Benjaree,Novelty Shop
Javier Caraballo,Novelty Shop
Akhtar Shahrestaani,Novelty Shop
Vanita Bhowmick,Novelty Shop
Nishant Menon,Novelty Shop
Josef Benda,Novelty Shop
Teresa Bender,Novelty Shop


In [3]:
/* Now Join Those to show customers we currently have in a SQL Server Database 
and the Category they qre in the External Table */

SELECT a.FullName, b.CustomerCategoryName
  FROM [WideWorldImporters].[Application].[People] a
  INNER JOIN [WideWorldImporters].[dbo].[web_customers_hdfs] b  ON a.FullName = b.PrimaryContact;
  GO

(663 rows affected)

Total execution time: 00:00:02.1527237

FullName
Aahlada Thota
Aakarsha Nookala
Aakriti Bhamidipati
Aakriti Byrraju
Aamdaal Kamasamudram
Abel Spirlea
Abel Tatarescu
Abhaya Rambhatla
Abhoy PrabhupÄda
Abhra Ganguly


## Next Steps: Continue on to Working with the SQL Server Data Pool